# Введение в обработку естественного языка

## Урок 7. Сверточные нейронные сети для анализа текста

### Задание:

Берем отзывы за лето (из архива с материалами или предыдущего занятия)

* Учим conv сеть для классификации
* Рассмотреть 2-а варианта сеточек:
  * Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
  * Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
* Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import gensim
import re
import tensorflow as tf
import keras
import nltk
import keras.backend as K

from pathlib import Path
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt", quiet=True)

True

In [2]:
# Функция подсчета f1_score
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Загрузка данных

In [3]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [4]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [6]:
df_train

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08
...,...,...,...
13836,4,Все нравится,2017-07-29
13837,5,Очень смешно программа пугается рута :),2017-07-28
13838,1,Не могу скачать ошибка номер 24,2017-08-06
13839,5,Сбербанк всегда рядом,2017-08-12


## Предобработка данных

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [8]:
# Готовим словарь
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [9]:
# Отфильтруем и возьмем только топ N токенов
tokens_filtered = [word for word in tokens if word.isalnum()] #фильтруем, чтобы остались только буквы и цифры

# Создаем словарь токен:количество в корпусе
dist = FreqDist(tokens_filtered)
dist

FreqDist({'приложение': 4123, 'удобно': 2201, 'работать': 1288, 'удобный': 1182, 'отлично': 860, 'нравиться': 763, 'хороший': 681, 'отличный': 677, 'телефон': 627, 'супер': 540, ...})

In [10]:
max_words = 200

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [11]:
# Создаем словарь из топ-200 токенов: токен:его рейтинг(место в топ-200)
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
# vocabulary

Подготовим Х и у к обучению. На входной Embedding слой модели (x_train и x_test) поступают положительные целые числа.

In [12]:
max_len = 40

# Преобразуем трейновый и тестовый датасеты:
# Приводим к нижнему регистру, токенизируем, удаляем символы, не являющиеся цифрами или буквами
# Проверяем: если токен есть в словаре, то записываем его номер из словаря в результат
# в противном случае - пропускаем, до полной длины добиваем нулями
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen - len(result))
    return padding + result[-maxlen:]

In [13]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ..., 164,  27,  84],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]])

Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием.

In [14]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [15]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4], dtype=int64)

In [16]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

## Keras CONV модель с Embedding слоем по умолчанию

In [17]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len)) 
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [19]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 3s 78ms/step - loss: 1.2421 - get_f1: 0.2939 - val_loss: 1.0015 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 2s 66ms/step - loss: 0.8949 - get_f1: 0.7206 - val_loss: 0.8087 - val_get_f1: 0.7429
Epoch 3/20
25/25 [==============================] - 2s 68ms/step - loss: 0.7335 - get_f1: 0.7561 - val_loss: 0.7285 - val_get_f1: 0.7505
Epoch 4/20
25/25 [==============================] - 2s 76ms/step - loss: 0.6886 - get_f1: 0.7689 - val_loss: 0.7163 - val_get_f1: 0.7560
Epoch 5/20
25/25 [==============================] - 2s 68ms/step - loss: 0.6704 - get_f1: 0.7732 - val_loss: 0.7078 - val_get_f1: 0.7566
Epoch 6/20
25/25 [==============================] - 2s 66ms/step - loss: 0.6578 - get_f1: 0.7782 - val_loss: 0.7015 - val_get_f1: 0.7581
Epoch 7/20
25/25 [==============================] - 2s 67ms/step - loss: 0.6482 - get_f1: 0.7815 - val_loss: 0.6993 - val_get_f1: 0.7604
Epoch 8/20
25/25 [=======================

In [20]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 18ms/step - loss: 0.6620 - get_f1: 0.7771


Test loss: 0.6619693040847778
Test f1_score: 0.7770693898200989


## Keras CONV модель с предобученным Embedding слоем

Предобученные векторы можно скачать [здесь](http://vectors.nlpl.eu/repository/20/180.zip).

In [21]:
# Загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True).vectors 

In [22]:
# Обрежем word_vectors по размеру эмбеддингов предыдущей задачи
word_vectors_matrix = [word_vectors[i][:128] for i in range(200)]
word_vectors_matrix[:3]

[array([ 1.0967804e+00, -2.2944486e+00,  1.9791678e+00,  3.4804371e-01,
         4.0753922e-01,  1.3286122e+00, -9.3368673e-01,  5.4947221e-01,
        -6.8077618e-01, -7.4963701e-01, -8.0936104e-02,  6.5788299e-02,
        -8.8564938e-01,  5.0793958e-01, -1.0864110e+00, -4.3317631e-01,
         2.0482888e-02, -5.7119979e-03, -1.0036458e+00,  3.1729680e-01,
         1.1956499e+00,  1.0685917e+00, -8.9309484e-01,  7.4419886e-01,
         4.4171312e-01, -2.0080043e-01, -2.6623638e+00,  1.7598321e-01,
        -2.0019765e+00, -5.6796205e-01, -2.0350738e-01, -7.3997623e-01,
         8.0226004e-01,  1.4174094e+00,  1.0990121e-01,  1.2313192e+00,
         1.6655500e+00, -2.6187131e-01,  1.7604357e+00, -3.1903556e-01,
         2.4266930e+00, -1.2458172e+00,  1.0440445e+00,  2.7978971e+00,
        -8.2489556e-01,  4.3603179e-01,  9.2311478e-01, -1.8360819e-01,
        -6.3582733e-02,  2.9669294e-01,  5.6388801e-01,  2.5535300e+00,
         2.3046949e+00, -1.8354168e+00, -6.8992484e-01, -1.38589

In [23]:
# Инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [25]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 3s 83ms/step - loss: 1.3161 - get_f1: 0.3627 - val_loss: 0.9901 - val_get_f1: 0.7144
Epoch 2/20
25/25 [==============================] - 2s 91ms/step - loss: 0.9065 - get_f1: 0.7199 - val_loss: 0.8611 - val_get_f1: 0.7234
Epoch 3/20
25/25 [==============================] - 2s 80ms/step - loss: 0.8328 - get_f1: 0.7326 - val_loss: 0.8222 - val_get_f1: 0.7335
Epoch 4/20
25/25 [==============================] - 2s 82ms/step - loss: 0.7842 - get_f1: 0.7386 - val_loss: 0.7946 - val_get_f1: 0.7363
Epoch 5/20
25/25 [==============================] - 2s 78ms/step - loss: 0.7498 - get_f1: 0.7479 - val_loss: 0.7807 - val_get_f1: 0.7412
Epoch 6/20
25/25 [==============================] - 2s 83ms/step - loss: 0.7218 - get_f1: 0.7511 - val_loss: 0.7578 - val_get_f1: 0.7382
Epoch 7/20
25/25 [==============================] - 2s 78ms/step - loss: 0.7010 - get_f1: 0.7549 - val_loss: 0.7537 - val_get_f1: 0.7435
Epoch 8/20
25/25 [=======================

In [26]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 21ms/step - loss: 0.7077 - get_f1: 0.7547


Test loss: 0.7076925039291382
Test f1_score: 0.7546936869621277


## Выводы:

Нейронная сеть без инициализации дает лучшую метрику (f1_score = 0.7770693898200989), нежели предобученная сетка (f1_score = 0.7546936869621277).